In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [3]:
train_df = pd.read_csv('../ass5/train.csv')
val_df = pd.read_csv('../ass5/validation.csv')
test_df = pd.read_csv('../ass5/test.csv')

In [4]:
print("Validation columns:", val_df.columns)
print("Test columns:", test_df.columns)
print("Validation size:", len(val_df))
print("Test size:", len(test_df))

Validation columns: Index(['sentence'], dtype='object')
Test columns: Index(['sentence'], dtype='object')
Validation size: 154
Test size: 156


In [5]:
train_sentences = train_df['sentence'].astype(str).tolist()
val_sentences = val_df['sentence'].astype(str).tolist()
test_sentences = test_df['sentence'].astype(str).tolist()


In [6]:
vectorizer = TfidfVectorizer()
val_tfidf = vectorizer.fit_transform(val_sentences)
test_tfidf = vectorizer.fit_transform(test_sentences)

In [7]:
val_sim_matrix = cosine_similarity(val_tfidf)
test_sim_matrix = cosine_similarity(test_tfidf)

In [8]:
np.fill_diagonal(val_sim_matrix, -1)
np.fill_diagonal(test_sim_matrix, -1)

In [9]:
val_nn_idx = val_sim_matrix.argmax(axis=1)
test_nn_idx = test_sim_matrix.argmax(axis=1)

In [10]:
val_df['nearest_neighbor_idx'] = val_nn_idx
val_df['nearest_neighbor_sentence'] = [val_sentences[i] for i in val_nn_idx]
test_df['nearest_neighbor_idx'] = test_nn_idx
test_df['nearest_neighbor_sentence'] = [test_sentences[i] for i in test_nn_idx]

In [13]:
print(val_df[['sentence', 'nearest_neighbor_sentence']].head())

                                            sentence  \
0  જે ધ્યાને લેતા દેશભરમાં નવીનતમ ટેકનોલોજીનો ઉપય...   
1                                  11. આ રમત "સાપની"   
2  એક માદા સ્વપ્ન પુસ્તક એવી સ્ત્રીને પ્રિય પ્રણય...   
3  કોર્ટે એવું અવલોકન કર્યુ છે કે પ્રથમદર્શનીય રી...   
4     આને લીધે આ સ્મારક શૂન્ય ઊર્જા પ્રોજેક્ટ બને છે   

                           nearest_neighbor_sentence  
0                              મિથેનોલ આવ્યો ક્યાંથી  
1  જે ધ્યાને લેતા દેશભરમાં નવીનતમ ટેકનોલોજીનો ઉપય...  
2  ગુજરાતે આ ઇન્ડેક્ષમાં જુદા જુદા પેરામીટર્સના જ...  
3  બાળકોની આગળની સાયકલ સીટને 15 કિલો વજનવાળા બાળક...  
4  રજાઓ અને સપ્તાહના અંતે, ઘણાં દેશ, માછીમારી, પ્...  


In [ ]:
val_df.to_parquet("validation_with_neighbors.parquet", index=False)
test_df.to_parquet("test_with_neighbors.parquet", index=False)